In [ ]:
print("Hello, World!")

In [ ]:
! pip install ollama
! pip install pandas
! pip install matplotlib
! ollama pull gemma3:4b 

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma3:4b', messages=[
  {
    'role': 'user',
    'content': 'In which year the programming language Java was introduced?',
  }
],
  options={"temperature":0.7}
)
print(response['message']['content'])


In [ ]:
!pip install llama-index llama-index-llms-ollama llama-index-embeddings-ollama pymupdf

In [ ]:
import glob
from pathlib import Path

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# --- 1) Locate the source code ---
resources_dir = Path("../src/main/java/org/example/")
java_paths = sorted(glob.glob(str(resources_dir / "*.java")))
if not java_paths:
    raise FileNotFoundError("No Java files found in ./input. Please place one there.")

# --- 2) Configure Ollama LLM and embedding model ---
Settings.llm = Ollama(model="gemma3:4b", request_timeout=120.0)
Settings.embed_model = OllamaEmbedding(model_name="embeddinggemma")

# --- 3) Load and index the Java code with embeddings ---
docs = SimpleDirectoryReader(input_files=[pdf_paths[0]]).load_data()
index = VectorStoreIndex.from_documents(docs)

# --- 4) Run a query ---
query_engine = index.as_query_engine(similarity_top_k=3)
prompt = "Build three test cases to improve the mutation score of Main.java."
response = query_engine.query(prompt)

print(f"Queried file: {Path(pdf_paths[0]).name}")
print("\n=== Response ===\n")
print(str(response))

# --- 5) Append the test cases to the original file ---

# Path to your test file
test_file_path = Path("../src/test/java/org/example/MainTest.java")

# Read the existing test file
with open(test_file_path, "r", encoding="utf-8") as f:
    test_content = f.read()

# Extract model output as text
new_tests = str(response).strip()

# Basic sanity: ensure the model didn’t include extra import/class definitions
# If it only includes methods, we’ll inject them before the final closing brace
if "class " not in new_tests and "}" in test_content:
    # Insert the test cases before the last closing brace
    test_content = test_content.rstrip()
    test_content = test_content[:-1] + "\n\n    " + new_tests.replace("\n", "\n    ") + "\n}"
else:
    # If the response includes a full class definition, just append it at the end
    test_content += "\n\n" + new_tests

# Write the updated test file back
with open(test_file_path, "w", encoding="utf-8") as f:
    f.write(test_content)

print(f"✅ Appended new test cases to: {test_file_path}")

In [ ]:
!pip install ollama pillow

In [ ]:
# Describe an image with Gemma3:4b via Ollama


import ollama
from PIL import Image

# --- 1) Path to your image ---
image_path = "./input/rag_simple.png"  # <-- replace with your image

# Optional: preview image in the notebook
display(Image.open(image_path))

# --- 2) Send prompt + image to Ollama ---
response = ollama.chat(
    model="gemma3:4b",
    messages=[
        {
            "role": "user",
            "content": "Given the image, build me an Agent AI workflow in Python using Ollama for testing Java software, including code coverage and mutation testing.",
            "images": [image_path],  # send the image to the model
        }
    ],
)

# --- 3) Print the description ---
print("=== Image Description ===")
print(response["message"]["content"])